## Building Docker Artifacts

In [1]:
cd farmer

/home/jupyter/kubeflow-sdk-demo/farmer


In [2]:
!docker login -u username -p password

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/jupyter/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [4]:
%%writefile config.py
gs_bucket_name="kubeflow-depankar-demo"
Bucket_uri="gs://kubeflow-depankar-demo"
version=1
store_artifacts=Bucket_uri + "/" + str(version)
data_path=Bucket_uri + "/" + "data/data_raw.csv"
processed_data=Bucket_uri + "/" + "processed/data_processed.csv"

Overwriting config.py


In [54]:
%%writefile Dockerfile
FROM tensorflow/tensorflow
ARG DEBIAN_FRONTEND=noninteractive
# Install apt dependencies
RUN apt-get update && apt-get install -y \
    git \
    gpg-agent \
    python3-cairocffi \
    protobuf-compiler \
    python3-pil \
    python3-lxml \
    python3-tk \
    wget
# Install gcloud and gsutil commands
# https://cloud.google.com/sdk/docs/quickstart-debian-ubuntu
#RUN apt-get update && apt-get install -y lsb-release && apt-get clean all
RUN echo "deb https://packages.cloud.google.com/apt cloud-sdk main" | tee -a /etc/apt/sources.list.d/google-cloud-sdk.list && \
    curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add - && \
    apt-get update -y && apt-get install google-cloud-sdk -y
WORKDIR /pipeline
COPY ./ ./
#RUN pip install -r requirements.txt
RUN /usr/bin/python3 -m pip install --upgrade pip
RUN pip install google-cloud-storage
RUN pip install google-cloud
RUN pip install gcsfs
RUN pip install dask[dataframe]
RUN pip install google-api-python-client
RUN pip install matplotlib
RUN pip install seaborn
RUN pip install pandas
RUN pip install sklearn
RUN pip install wget
#RUN pip install dvc
RUN pip install "dask[dataframe]" --upgrade
ENV TF_CPP_MIN_LOG_LEVEL 3

Overwriting Dockerfile


## Build Docker Image

In [55]:
!docker build -t depankars/kubeflow-sdk-farmer .

Sending build context to Docker daemon  27.65kB
Step 1/19 : FROM tensorflow/tensorflow
 ---> b51f642475ab
Step 2/19 : ARG DEBIAN_FRONTEND=noninteractive
 ---> Using cache
 ---> e2b9630a5545
Step 3/19 : RUN apt-get update && apt-get install -y     git     gpg-agent     python3-cairocffi     protobuf-compiler     python3-pil     python3-lxml     python3-tk     wget
 ---> Using cache
 ---> 9f6f79a076a4
Step 4/19 : RUN echo "deb https://packages.cloud.google.com/apt cloud-sdk main" | tee -a /etc/apt/sources.list.d/google-cloud-sdk.list &&     curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add - &&     apt-get update -y && apt-get install google-cloud-sdk -y
 ---> Using cache
 ---> 16aa13e5b9f8
Step 5/19 : WORKDIR /pipeline
 ---> Using cache
 ---> 145dfdc47e6f
Step 6/19 : COPY ./ ./
 ---> e87278efb138
Step 7/19 : RUN /usr/bin/python3 -m pip install --upgrade pip
 ---> Running in fced597bbe55
  Attempting uninstall: pip
    Found existing installation: pip 20.2.4
    Un

## Push Docker Image to Docker hub

In [56]:
!docker push depankars/kubeflow-sdk-farmer

Using default tag: latest
The push refers to repository [docker.io/depankars/kubeflow-sdk-farmer]

231dfd28: Preparing 
d9c2d84f: Preparing 
899e1d08: Preparing 
e6ca7c94: Preparing 
5015742d: Preparing 
dfbd6969: Preparing 
2ddf8d1a: Preparing 
a8013090: Preparing 
3f5ad6cc: Preparing 
4a2dc57b: Preparing 
976a1b86: Preparing 
77227b48: Preparing 
03a8310a: Preparing 
ef00e0cf: Preparing 
7bcdea39: Preparing 
984157d1: Preparing 
8d4da588: Preparing 
f6d04422: Preparing 
45155a19: Preparing 
b1a4d7c3: Preparing 
af4dda95: Preparing 
6741f73b: Preparing 
2698ea9f: Preparing 
bcdea39: Pushed   611.5MB/585.5MBtensorflow 20Alatest: digest: sha256:0f9f2ce3e57b14e9e084b9ded5711ac66d938fc38e87fe1079c27f89887bb907 size: 5365


## Install Kubeflow SDk

In [8]:
!pip install kfp

     |████████████████████████████████| 298 kB 6.7 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 127 kB 83.8 MB/s            
     |████████████████████████████████| 636 kB 82.5 MB/s            
     |████████████████████████████████| 1.6 MB 87.9 MB/s            
     |████████████████████████████████| 61 kB 38 kB/s              
     |████████████████████████████████| 152 kB 62.3 MB/s            
     |████████████████████████████████| 54 kB 4.1 MB/s             
     |████████████████████████████████| 52 kB 2.3 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 56 kB 4.7 MB/s             
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     |████████████████████████████████| 87 kB 9.9 MB/s             
  Preparing metadata (setup.py) ... done
     |███████████

# Building Kubeflow Pipeline

In [11]:
import kfp
import kfp.components as comp
from kubernetes.client.models import V1EnvVar

## Define kubeflow pipeline Component

In [57]:
@kfp.dsl.component
def get_data():
    # Defining component configuration
    getdata_component = kfp.dsl.ContainerOp(
        name='Data-Preparation',
        image='docker.io/depankars/kubeflow-sdk-farmer',
        command=['python', 'get_data.py'],
        )
    return getdata_component

In [58]:
@kfp.dsl.component
def data_preprocessing():    
    # Defining component configuration
    data_preprocessing = kfp.dsl.ContainerOp(
        name='data-preprocessing',
        image='docker.io/depankars/kubeflow-sdk-farmer',
        command=['python', 'process_data.py'],
        )
    return data_preprocessing

In [59]:
@kfp.dsl.component
def training():
    # Defining component configuration
    training_component = kfp.dsl.ContainerOp(
        name='training',
        image='docker.io/depankars/kubeflow-sdk-farmer',
        command=['python', 'train.py'],
        file_outputs={'mlpipeline-ui-metadata':'/mlpipeline-ui-metadata.json', "mlpipeline-metrics":'/mlpipeline-metrics.json'}
        )
    
    return training_component

In [60]:
# Let see output of component configuration
debug = True
if debug :
    training_component_vis = training()
    print(training_component_vis)

{'ContainerOp': {'is_exit_handler': False, 'human_name': 'training', 'display_name': None, 'name': 'training 800007fa4dc6f649', 'node_selector': {}, 'volumes': [], 'tolerations': [], 'affinity': {}, 'pod_annotations': {}, 'pod_labels': {}, 'num_retries': 0, 'retry_policy': None, 'backoff_factor': None, 'backoff_duration': None, 'backoff_max_duration': None, 'timeout': 0, 'init_containers': [], 'sidecars': [], 'loop_args': None, '_component_spec_inputs_with_pipeline_params': [], '_inputs': [], 'dependent_names': [], 'enable_caching': True, 'attrs_with_pipelineparams': ['node_selector', 'volumes', 'pod_annotations', 'pod_labels', 'num_retries', 'init_containers', 'sidecars', 'tolerations', '_container', 'artifact_arguments', '_parameter_arguments'], '_is_v2': False, '_container': {'args': None,
 'command': ['python', 'train.py'],
 'env': None,
 'env_from': None,
 'image': 'docker.io/depankars/kubeflow-sdk-farmer',
 'image_pull_policy': None,
 'lifecycle': None,
 'liveness_probe': None,
 

## Final Kubeflow pipeline Definition

In [61]:
@kfp.dsl.pipeline(
  name="Modeling Swiss farmer's attitudes about",
  description="Modeling Swiss farmer's attitudes about"
)
def farmer():
    download_data = get_data()
    download_data.execution_options.caching_strategy.max_cache_staleness = "P0D"
    data_processing = data_preprocessing().after(download_data)
    data_processing.execution_options.caching_strategy.max_cache_staleness = "P0D"
    train = training().after(data_processing)
    train.execution_options.caching_strategy.max_cache_staleness = "P0D"


In [62]:
# Let see output of pipeline configuration
debug = True
if debug :
    training_pipeline_output = farmer()
    print(training_pipeline_output)

None


## Compile Kubeflow Pipeline 
* It will Generate .zip file inside this contain YAMl file which contain the configuration of kubeflow pipeline

In [63]:
kfp.compiler.Compiler().compile(farmer, 'farmer-kubeflow-sdk-demo.zip')

## Connect to deployed kubeflow pipeline Endpoint (GCP)

In [64]:
# Create kfp client
# Note: Add the KubeFlow Pipeline endpoint below if the client is not running on the same cluster.
client = kfp.Client("https://4cc472dcfd645f-dot-us-central1.pipelines.googleusercontent.com")

### Create Experiment 

In [65]:
EXPERIMENT_NAME = 'farmer-3'
experiment = client.create_experiment(name=EXPERIMENT_NAME)

### Deploy pipeline to kubeflow pipeline Endpoint

In [66]:
run = client.run_pipeline(experiment.id, 'farmer-run-1', 'farmer-kubeflow-sdk-demo.zip')